# Neural net js

> The goal of this project is implement a neural net in javascript that is as easy to understand as possible.

This project will also look at using an [nbdev](https://github.com/fastai/nbdev/) style of library development using [tslab](https://github.com/yunabe/tslab) to provide a js kernel.

## Live demo

This [gist via bl.ocks](https://bl.ocks.org/pete88b/2aa60d189006bba7c59039f1e9d55936) shows how a model, learning in browser, converges with a 3d scatter plot.

# Quick example

Let's train a classifier using [iris.data](https://archive.ics.uci.edu/ml/datasets/iris).

In [1]:
import {argmax} from './src/util.module.js';
import {head,tail,parseCsv,IRIS_CLASS_MAP,IrisRowHandler} from './src/data.module.js';
import {BinaryCrossEntropyLoss,Linear,Sigmoid,ReLU,Learner} from './src/nn.module.js';

In [2]:
let stringData=require('fs').readFileSync('data/iris.data').toString();
let data=parseCsv(stringData, new IrisRowHandler()).result;
let lossFn=new BinaryCrossEntropyLoss();
let model=[new Linear(4,50), new ReLU(), new Linear(50,3), new Sigmoid()];
let learn=new Learner(model, lossFn, data);
learn.fit(25);

epoch -1 valid loss 0.5903966819442324 metrics [ 0.5666666666666667 ]
epoch 0 valid loss 0.3407337096444165 metrics [ 0.7333333333333333 ]
epoch 1 valid loss 0.2862552656446179 metrics [ 0.7666666666666667 ]
epoch 2 valid loss 0.25977964626282773 metrics [ 0.8333333333333334 ]
epoch 3 valid loss 0.2425962174061848 metrics [ 0.8333333333333334 ]
epoch 4 valid loss 0.23052460534422084 metrics [ 0.8666666666666667 ]
epoch 5 valid loss 0.2219764850003557 metrics [ 0.8666666666666667 ]
epoch 6 valid loss 0.2155868764802235 metrics [ 0.8666666666666667 ]
epoch 7 valid loss 0.2100849828904428 metrics [ 0.8666666666666667 ]
epoch 8 valid loss 0.20451255769038718 metrics [ 0.8666666666666667 ]
epoch 9 valid loss 0.19991713342679168 metrics [ 0.8666666666666667 ]
epoch 10 valid loss 0.19631987142339363 metrics [ 0.8666666666666667 ]
epoch 11 valid loss 0.19247678235149668 metrics [ 0.8666666666666667 ]
epoch 12 valid loss 0.18867427254256755 metrics [ 0.8666666666666667 ]
epoch 13 valid loss 0.1

We can look at predictions our trained model makes on the validation data.

For each row, `learn.predict` gives us `[preds, predicted label, actual label]`

In [3]:
let preds=learn.predict(learn.xValid, learn.yValid, (y=>`${argmax(y)}: ${IRIS_CLASS_MAP[argmax(y)]}`));
tail(preds,3);

-3 [
  [ 0.00061238812657286, 0.19764246030216065, 0.8763671384120886 ],
  '2: Iris-virginica',
  '2: Iris-virginica'
]
-2 [
  [ 0.0034785677047756308, 0.2603165853376491, 0.7657497145240926 ],
  '2: Iris-virginica',
  '2: Iris-virginica'
]
-1 [
  [ 0.9928633203666908, 0.014137138112522535, 0.00435879637193152 ],
  '0: Iris-setosa',
  '0: Iris-setosa'
]


and we can easily make up some data of our own and see what the model predicts

In [4]:
let rh=new IrisRowHandler();
rh.handleRow('5.5,3.5,10.4,0.2,Iris-setosa');
rh.handleRow('5.5,2.6,0.4,1.2,Iris-versicolor');
rh.handleRow('0.5,3.0,5.1,1.8,Iris-virginica');
learn.predict(...rh.result, (y=>`${argmax(y)}: ${IRIS_CLASS_MAP[argmax(y)]}`));

[
  [
    [ 0.1230832698012669, 0.020098951596435544, 0.014637429800718547 ],
    '0: Iris-setosa',
    '0: Iris-setosa'
  ],
  [
    [ 0.2650451679496162, 0.44993355141353014, 0.135656747558312 ],
    '1: Iris-versicolor',
    '1: Iris-versicolor'
  ],
  [
    [
      0.9766828277236442,
      0.00001019703456778939,
      0.000003005658805141498
    ],
    '0: Iris-setosa',
    '2: Iris-virginica'
  ]
]


## Setting up a development environment

This project uses the [tslab Dockerfile for running on mybinder.org](https://github.com/yunabe/tslab-examples/blob/master/Dockerfile_prebuilt).

```
git clone https://github.com/pete88b/neuralnet_js.git
cd neuralnet_js
docker build -t neuralnet_js .
docker run -d -p 8888:8888 -p 8000:8000 --mount type=bind,source="<absolute path to neuralnet_js on your machine>",target=/home/node/tslab-examples neuralnet_js
```
Check the container logs for a section that looks a bit like;
```
To access the notebook, open this file in a browser:
file:///home/node/.local/share/jupyter/runtime/nbserver-1-open.html
Or copy and paste one of these URLs:
http://4f1ee683b96c:8888/?token=b3dd4ed644617cfb795dd8eb899aafea4c2168d8dc897357
or http://127.0.0.1:8888/?token=b3dd4ed644617cfb795dd8eb899aafea4c2168d8dc897357
```
Copy the `127.0.0.1` URL and paste it into your browser.

Note: The `docker run` command above uses 2 ports;
- 8888 for jupyter and
- 8000 which we can use to view demo web pages with http://localhost:8000/demo/
    - Run `python3 -m http.server 8000` from a jupyter terminal to start the http server
    - we manually start the server so that we don't have to change Dockerfile

If you'd like to work with the tslab-examples, just docker run without the --mount;
```
docker run -d -p 8888:8888 neuralnet_js
```

## Converting notebooks to javascript files

From a jupyter terminal you can;

`$ python3 nbdev_js.py`
```
Converting 00_testutil.ipynb to src/testutil.js
Converting 00_testutil.ipynb to src/testutil.module.js
...
Converting index.ipynb to README.md
```

Note: I've added a local file `mk` so I only have to type `./mk` at the terminal.

See: `99_nbdev_js.ipynb` for notebook conversion details.